In [1]:
from streamlit_app import *

In [2]:
import datetime
from logging import exception
from typing import List

import gspread
import numpy as np
import pandas as pd
import plotly.express as px
import streamlit as st
from google.oauth2 import service_account
import matplotlib.colors as mcolors

In [3]:
sheet_url = st.secrets["private_gsheets_url"]
df = get_worksheet(sheet_url, 0)

2022-04-14 18:17:38.188 
  command:

    streamlit run /home/wotashu/.cache/pypoetry/virtualenvs/streamlit-cheki-5kyQcRg0-py3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [4]:
cheki_df = get_worksheet(sheet_url, 0)
idol_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet(sheet_url, 2)

In [5]:
venue_df[venue_df['location'].str.contains("PlatinumGarage")]

,location,neighborhod,city,prefecture,country
121,PlatinumGarage西新宿スタジオ,本町,渋谷区,東京都,日本


In [6]:
cheki_df['cheki_id'] = cheki_df.index

In [19]:
name_cols = [i for i in cheki_df.columns if "name" in i]

In [20]:
cheki_df['idol_count'] = cheki_df[name_cols].replace("", np.nan).count(axis=1)

In [9]:
cheki_df.replace("", np.nan, inplace=True)

In [11]:
merged_df = pd.merge(cheki_df, venue_df, how="left", left_on="location", right_on="location")

In [13]:
group_location = pd.DataFrame(merged_df.groupby("location")["cheki_id"].count().sort_values(ascending=False)).reset_index()

In [14]:
group_location[:20]

,location,cheki_id
0,新宿HEADPOWER,520
1,Tokyo Link's Hall,404
2,Cosmic Lab,367
3,郵送,257
4,池袋リヴォイス,160
5,池袋SOUND PEACE,159
6,Space Odd,94
7,新宿アルタKeyStudio,89
8,Deseo Mini,85
9,赤羽レニーAlpha,78


In [16]:
group_location['location']

0                    新宿HEADPOWER
1              Tokyo Link's Hall
2                     Cosmic Lab
3                             郵送
4                        池袋リヴォイス
                 ...            
175    Bellesalle Shibuya Garden
176                     ベルサール西新宿
177        PlatinumGarage西新宿スタジオ
178                       池袋AKビル
179                   Nadya Park
Name: location, Length: 180, dtype: object

In [17]:
cheki_df.columns = [col.lower() for col in cheki_df.columns]

In [21]:
cheki_df

,note,location,date,name1,name2,name3,name4,name5,name6,name7,name8,cheki_id,idol_count
1,NaN,Bellesalle Shibuya Garden,2013-05-05,矢島舞美@℃-ute,中島早貴@℃-ute,鈴木愛理@℃-ute,岡井千聖@℃-ute,萩原舞@℃-ute,NaN,NaN,NaN,1,5
2,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,AIRI@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
3,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,SAKURA@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1
4,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,YUUKA@Aither,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1
5,NaN,Best Western Plus Silverdale Beach Hotel,2014-06-21,木村美咲@ラストアイドル,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5128,NaN,Cosmic Lab,2022-04-10,Suzy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5128,1
5129,NaN,Cosmic Lab,2022-04-10,Suzy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5129,1
5130,NaN,Cosmic Lab,2022-04-10,Suzy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5130,1
5131,NaN,Cosmic Lab,2022-04-10,Suzy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5131,1


In [86]:
names = {}
name_cols = [col for col in cheki_df.columns if "name" in col]
for index, row in cheki_df.iterrows():
    names[index] = set([row[col] for col in name_cols if row[col] is not np.NaN])

In [87]:
cheki_df['names'] = cheki_df.apply(lambda x: [x[col] for col in name_cols if x[col] is not np.NaN], axis=1)

In [97]:
df2 = cheki_df[['cheki_id', 'names']]

In [98]:
df2

,cheki_id,names
1,1,"[矢島舞美@℃-ute, 中島早貴@℃-ute, 鈴木愛理@℃-ute, 岡井千聖@℃-ut..."
2,2,[AIRI@Aither]
3,3,[SAKURA@Aither]
4,4,[YUUKA@Aither]
5,5,[木村美咲@ラストアイドル]
...,...,...
4439,4439,[Suzy]
4440,4440,[Suzy]
4441,4441,[Suzy]
4442,4442,[Suzy]


In [109]:
df3 = pd.get_dummies(
    df2.explode('names'),
    columns=['names'],
    # prefix="", prefix_sep=""
).groupby('cheki_id').sum().reset_index()

In [110]:
new_df = pd.merge(cheki_df[['cheki_id','location', 'date']], df3, how='left', right_on='cheki_id', left_on='cheki_id')

In [116]:
pd.DataFrame(new_df[[col for col in new_df.columns if "names_" in col]].sum().sort_values(ascending=False))

,0
names_Suzy,947
names_恵深あむ@メルティハート,404
names_瀬乃悠月@恒星宇宙,272
names_七瀬千夏@たまご姫【たまプリ】,272
names_楠木りほ@天使突抜ニ読ミ,217
...,...
names_古川未鈴@でんぱ組.inc,1
names_和ちひろ@忍者クライシス,1
names_土井千春@THAT's so,1
names_夏来彩世@プシュケとラーガ,1


In [120]:
new_df.groupby(['cheki_id', 'location', 'date']).sum()

,,,names_AIRI@Aither,names_Aarie@ACT102,names_Ale 愛礼@国際組,names_Ally@Seishun Youth Academy,names_Anthony@Seishun Youth Academy,names_Aoi@Finally,names_Beak@SwayEmotionsSlightly,names_Berry@ACT102,names_Candy馨xin@花リミ,names_Crystal@ACT102,...,names_響木涼介@恒星宇宙,names_餅宮りよ@81moment,names_高月圭@SPADES,names_高橋みあ@Youぱ〜む,names_高橋紫微@アイオケ,names_鳥越結貴@ヒップバーン,names_鳴上綺羅@光,names_黒ゑリズ@蜜兎,names_黒瀬瑠衣@あっしゅない！,names_𝐍𝐚𝐭𝐬𝐮𝐊𝐢@QpQueen
cheki_id,location,date,,,,,,,,,,,,,,,,,,,,,
1,Bellesalle Shibuya Garden,2013-05-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Best Western Plus Silverdale Beach Hotel,2014-06-21,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Best Western Plus Silverdale Beach Hotel,2014-06-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Best Western Plus Silverdale Beach Hotel,2014-06-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Best Western Plus Silverdale Beach Hotel,2014-06-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439,Cosmic Lab,2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4440,Cosmic Lab,2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4441,Cosmic Lab,2022-02-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
